In [1]:
from src.lexer import Lexer
from src.parser import Parser
from src.parser.utils import ast_print, asdict

In [2]:
primitives = ["none", "5", "4.69", "'hello world'", "true", "false", "x"]
for primitive in primitives:
    lexer = Lexer(primitive)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_primary()
    print(node)

NoneNode()
NumberNode(value=5, type=NumberType.INTEGER)
NumberNode(value=4.69, type=NumberType.FLOAT)
StringNode(value='hello world')
BooleanNode(value=True)
BooleanNode(value=False)
IdentifierNode(name='x')


---

In [3]:
lines = [
    "some_list[0]", "x[0][1]", "x[index]", # postfixes
    "not true", "-3", "+2.78", # unary operators
    "human.name", "node.value.type", # attributes
    "- human.friends[2].age", # mixed up
]
for code in lines:
    lexer = Lexer(code)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_unary_or_primary()
    print(node)

IndexNode(collection=IdentifierNode(name='some_list'), index=NumberNode(value=0, type=NumberType.INTEGER))
IndexNode(collection=IndexNode(collection=IdentifierNode(name='x'), index=NumberNode(value=0, type=NumberType.INTEGER)), index=NumberNode(value=1, type=NumberType.INTEGER))
IndexNode(collection=IdentifierNode(name='x'), index=IdentifierNode(name='index'))
UnaryOpNode(operator=UnaryOperatorType.NOT, operand=BooleanNode(value=True))
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=NumberNode(value=3, type=NumberType.INTEGER))
UnaryOpNode(operator=UnaryOperatorType.POSITIVE, operand=NumberNode(value=2.78, type=NumberType.FLOAT))
AttributeNode(object=IdentifierNode(name='human'), attribute='name')
AttributeNode(object=AttributeNode(object=IdentifierNode(name='node'), attribute='value'), attribute='type')
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=AttributeNode(object=IndexNode(collection=AttributeNode(object=IdentifierNode(name='human'), attribute='friends'), ind

---

In [4]:
expressions = [
    "5", " - (- 5)",
    "[1, 2, 3]", "['a', 'b', 'c']",
    "{'a': 1, 'b': 2}",
]
for code in expressions:
    lexer = Lexer(code)
    tokens = lexer.tokenize()
    parser = Parser(tokens)
    node = parser.parse_expression()
    print(node)

NumberNode(value=5, type=NumberType.INTEGER)
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=NumberNode(value=5, type=NumberType.INTEGER)))
ListNode(elements=[NumberNode(value=1, type=NumberType.INTEGER), NumberNode(value=2, type=NumberType.INTEGER), NumberNode(value=3, type=NumberType.INTEGER)])
ListNode(elements=[StringNode(value='a'), StringNode(value='b'), StringNode(value='c')])
DictionaryNode(pairs=[(StringNode(value='a'), NumberNode(value=1, type=NumberType.INTEGER)), (StringNode(value='b'), NumberNode(value=2, type=NumberType.INTEGER))])


---

In [5]:
with open("tests/parser/expressions.txt", "r") as f:
    code = f.read()
    
print(code)

// literals
none
5
4.69
'hello world'
true
false
some_variable

// unary operations
-7
+4.5
not true

// postfixes (indexing, attributes)
some_list[47]
other_list[pre_index][post_index]

human.name
node.value.type

// mixed up
- human.friends[2].age

// lists
[1, 2, 3]
['a', 'b', 'c']
[1, 2, 3, [4, 5, 6]]

// dictionaries
{'a': 1, 'b': 2}
{'a': 1, 'b': 2, 'c': {'d': 3, 'e': 4}}


In [6]:
lexer = Lexer(code)
tokens = lexer.tokenize()

parser = Parser(tokens)
ast = parser.parse()
for stmt in ast.statements:
    print(stmt)

NoneNode()
NumberNode(value=5, type=NumberType.INTEGER)
NumberNode(value=4.69, type=NumberType.FLOAT)
StringNode(value='hello world')
BooleanNode(value=True)
BooleanNode(value=False)
IdentifierNode(name='some_variable')
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, operand=NumberNode(value=7, type=NumberType.INTEGER))
UnaryOpNode(operator=UnaryOperatorType.POSITIVE, operand=NumberNode(value=4.5, type=NumberType.FLOAT))
UnaryOpNode(operator=UnaryOperatorType.NOT, operand=BooleanNode(value=True))
IndexNode(collection=IdentifierNode(name='some_list'), index=NumberNode(value=47, type=NumberType.INTEGER))
IndexNode(collection=IndexNode(collection=IdentifierNode(name='other_list'), index=IdentifierNode(name='pre_index')), index=IdentifierNode(name='post_index'))
AttributeNode(object=IdentifierNode(name='human'), attribute='name')
AttributeNode(object=AttributeNode(object=IdentifierNode(name='node'), attribute='value'), attribute='type')
UnaryOpNode(operator=UnaryOperatorType.NEGATIVE, ope